# Compute UP08 ISS/Portcell model intersection with envelopes for GA VIS IR.

We don't use bounding boxes here, because the number of cells in the model is much more (about 1000 times) than number of GA envelopes. The complexity is not reduced significantly using bounding box.

In [2]:
import typing as tp

In [3]:
import json

In [43]:
from tqdm import tqdm

In [22]:
from pprint import pprint, pformat

In [28]:
from functools import reduce

In [4]:
import mckit as mk

In [7]:
from pathlib import Path
Path.cwd()

WindowsPath('D:/dvp/dev/mcnp/upp/wrk/up08/intersection')

In [5]:
upp08 = mk.read_mcnp("u200.i")

In [6]:
upp08_boxes  = []

for c in upp08:
    bb = c.shape.bounding_box(tol=10)
    print(c.name())
    upp08_boxes.append(bb)

len(upp08_boxes)

In [9]:
envelop = mk.read_mcnp("ga-vis-ir.i")

In [14]:
len(envelop)

5

envelop_boxes = []

for c in envelop:
    bb = c.shape.bounding_box(tol=10)
    print(c.name())
    envelop_boxes.append(bb)

In [12]:
envelop_union = reduce(mk.Body.union, envelop[:3]).simplify(min_volume=0.1)

In [48]:
envelopes_cells: tp.List[mk.Body] = []

for c8 in tqdm(upp08):
    new_cell = c8.intersection(envelop_union).simplify(min_volume=0.1)
    if not new_cell.shape.is_empty():
        envelopes_cells.append(new_cell)

len(envelopes_cells), len(upp08)

100%|██████████████████████████████████████| 4261/4261 [05:20<00:00, 13.28it/s]


(467, 4261)

In [49]:
envelopes_cells_names: tp.List[int] = list(map(mk.Body.name, envelopes_cells))

In [53]:
envelopes_cells_names[:]

[323253,
 323254,
 323255,
 323256,
 323257,
 323258,
 323259,
 323260,
 323261,
 323262,
 323263,
 323264,
 323265,
 323266,
 323267,
 323268,
 323269,
 323270,
 323271,
 323272,
 323273,
 323274,
 323275,
 323276,
 323277,
 323278,
 323279,
 323280,
 323281,
 323282,
 323283,
 323284,
 323285,
 323286,
 323287,
 323288,
 323289,
 323290,
 323291,
 323292,
 323293,
 323294,
 323295,
 323296,
 323297,
 323298,
 323299,
 323300,
 323301,
 323302,
 323304,
 323306,
 323307,
 323308,
 323309,
 323310,
 323311,
 323312,
 323313,
 323314,
 323315,
 323316,
 323317,
 323318,
 323319,
 323320,
 323321,
 323322,
 323323,
 323324,
 323325,
 323326,
 323327,
 323328,
 323329,
 323330,
 323331,
 323332,
 323333,
 323334,
 323335,
 323336,
 323337,
 323338,
 323339,
 323340,
 323341,
 323342,
 323343,
 323344,
 323345,
 323346,
 323347,
 323348,
 323349,
 323350,
 323351,
 323352,
 323353,
 323354,
 323355,
 323356,
 323357,
 323358,
 323359,
 323360,
 323361,
 323362,
 323363,
 323364,
 323365,
 

In [51]:
new_universe = mk.Universe(envelopes_cells, name_rule="clash")
new_universe.save("vis-ir-intersection-union.i")

In [ ]:
component_to_cell_map: tp.Dict[str, Set[int]] = dict()

In [ ]:
names_of_cells_with_materials: tp.Set[int] = {c.name() for c in envelopes_cells if c.material()}
component_to_cell_map["vis-ir-gdc", names_of_cells_with_materials]

In [ ]:
upp08_volumes = []
for c in upp08:
    if c.material():
        upp08_volumes.append(c.shape.volume(min_volume=1.e-3))
    else:
        upp08_volumes.append(0)

In [ ]:
def select_criteria(c: mk.Body):
    